In [1]:
!pip install numpy
!pip install opencv-python
!pip install matplotlib


[notice] A new release of pip is available: 23.0.1 -> 25.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 25.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [18]:
import numpy as np
import cv2 as cv
import glob
import matplotlib.pyplot as plt
from typing import Tuple

### Load Prepare calibration data ###

In [ ]:
@staticmethod
def prepare_calibration_data(directory_pattern: str, chessboard_size: Tuple[int, int]):
    """
    Prepare calibration data from images.
    
    Args:
        directory_pattern: glob pattern for images e.g., 'content/images/*.jpeg'
        chessboard_size: Inner corner dimensions (width, height)
        
    Returns:
        objpoints, imgpoints: Arrays for cv2.calibrateCamera()
    """
    # termination criteria
    criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)
    
    # prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
    objp = np.zeros((chessboard_size[1]*chessboard_size[0],3), np.float32)
    objp[:,:2] = np.mgrid[0:chessboard_size[0],0:chessboard_size[1]].T.reshape(-1,2)
    
    # Arrays to store object points and image points from all the images.
    objpoints = [] # 3d point in real world space
    imgpoints = [] # 2d points in image plane.
    
    images = glob.glob(directory_pattern)
    img_shape = None

    for fname in images:
        img = cv.imread(fname)
        gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
        
        if img_shape is None:
            img_shape = gray.shape[::-1]

        # Find the chess board corners
        ret, corners = cv.findChessboardCorners(gray, chessboard_size, None)
        
        # If found, add object points, image points (after refining them)
        if ret == True:
            objpoints.append(objp)
            corners2 = cv.cornerSubPix(gray,corners, (11,11), (-1,-1), criteria)
            imgpoints.append(corners2)
            
            # Draw and display the corners
            cv.drawChessboardCorners(img, chessboard_size, corners2, ret)
            cv.imshow('img', img)
            cv.waitKey(500)
    
    cv.destroyAllWindows()
    return objpoints, imgpoints, img_shape

In [33]:
chessboard_size = (9, 6)
objpoints, imgpoints, img_shape = prepare_calibration_data('content/images/*.jpeg', chessboard_size)

print(f"objpoints: {len(objpoints)}")
print(f"imgpoints: {len(imgpoints)}")

objpoints: 20
imgpoints: 20


### Camera Calibration ###

In [34]:
ret, mtx, dist, rvecs, tvecs = cv.calibrateCamera(objpoints, imgpoints, img_shape, None, None)